In [30]:
import os
import numpy as np
import pandas as pd

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action = 'ignore')

import pickle

import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 불러오기
ratings = pd.read_csv('/home/ryu/thesis/data/amazon/Amazon_ratings.csv')

cnt = ratings.groupby('user_id').count()['rating']
keys = cnt[cnt>3].keys()
ratings = ratings[ratings['user_id'].isin(keys)]

In [34]:
ratings.user_id.nunique() - len(black)

9631

In [3]:
# RMSE 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [33]:
# black sheep 불러오기
with open('/home/ryu/thesis/new_amazon/black_id.pkl', 'rb') as f:
    black = pickle.load(f)

In [29]:
state = [8, 15, 35, 42, 84]
black_all = []
black_only = []

for s in state:

    ratings = pd.read_csv('/home/ryu/thesis/data/amazon/Amazon_ratings.csv')

    cnt = ratings.groupby('user_id').count()['rating']
    keys = cnt[cnt>3].keys()
    ratings = ratings[ratings['user_id'].isin(keys)]

    # black sheep 제거한 test, train 만들기
    x = ratings.copy()
    y = ratings['user_id']
    ratings_train, ratings_test = train_test_split(x, test_size=0.25, stratify=y, random_state=s)

    black_train = ratings_train[ratings_train['user_id'].isin(black)]
    black_test = ratings_test[ratings_test['user_id'].isin(black)]

    ratings_train = ratings_train[~ratings_train['user_id'].isin(black)]
    ratings_test = ratings_test[~ratings_test['user_id'].isin(black)]
    ratings = ratings[~ratings['user_id'].isin(black)]

    # 전체 bestseller
    all_item_mean = ratings_train.groupby(['item_id'])['rating'].mean()

    all_rmse = []
    for user in set(black_test.index): # unique user_ids
        y_true = black_test.loc[user][['item_id', 'rating']] # ratings on a particular item (해당 사용자가 평가한 영화들의 평점)
        if black_test.loc[user]['item_id'] in all_item_mean.index:
            y_pred = all_item_mean[black_test.loc[user]['item_id']] # 해당 사용자가 평가한 영화들 평점의 평균 (best seller 추천이므로 예측값에 해당)
        else:
            y_pred = all_item_mean.mean()
        a_accuracy = RMSE(y_true['rating'], y_pred)
        all_rmse.append(a_accuracy)

    bestseller_RMSE = np.mean(all_rmse)

    # 실험 결과
    print(f'Bestseller RMSE: {bestseller_RMSE}')
    # 결과 저장
    black_all.append(bestseller_RMSE)

    # Black Sheep ONLY bestseller


    black_item_mean = black_train.groupby(['item_id'])['rating'].mean()

    blk_rmse = []
    for user in set(black_test.index): # unique user_ids
        y_true = black_test.loc[user][['item_id', 'rating']] # ratings on a particular item (해당 사용자가 평가한 영화들의 평점)
        if black_test.loc[user]['item_id'] in black_item_mean.index:
            y_pred = black_item_mean[black_test.loc[user]['item_id']] # 해당 사용자가 평가한 영화들 평점의 평균 (best seller 추천이므로 예측값에 해당)
        else:
            y_pred = black_item_mean.mean()
        b_accuracy = RMSE(y_true['rating'], y_pred)
        blk_rmse.append(b_accuracy)

    blk_bestseller_RMSE = np.mean(blk_rmse)

    # 실험 결과
    print(f'Black Bestseller RMSE: {blk_bestseller_RMSE}')
    # 결과 저장
    black_only.append(blk_bestseller_RMSE)

Bestseller RMSE: 0.7612261985444495
Black Bestseller RMSE: 0.8566528629382743
Bestseller RMSE: 0.7584417952083649
Black Bestseller RMSE: 0.8529407401197725
Bestseller RMSE: 0.7651581201251874
Black Bestseller RMSE: 0.8550422477132075
Bestseller RMSE: 0.7551237547513936
Black Bestseller RMSE: 0.8520294349100557
Bestseller RMSE: 0.7676760046804372
Black Bestseller RMSE: 0.8637268767520256
